<a href="https://colab.research.google.com/github/ashfaq-polit/Large_language_models/blob/master/LangGraph_ChatBot_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate bitsandbytes langchain langgraph langchain-community pydot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [ ]:
from google.colab import userdata
hf_token = userdata.get('llama3')  # stored securely in Colab secrets

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16,
    device_map="auto"
)

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True
)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from langchain_community.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=llama_pipeline)

In [ ]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import re

# Define state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Tool (manually called)
def get_stock_price(symbol: str) -> float:
    return {
        "MSFT": 200.3,
        "AAPL": 100.4,
        "AMZN": 150.0,
        "RIL": 87.6
    }.get(symbol.upper(), 0.0)


In [ ]:
def chatbot_node(state: State) -> State:
    user_message = state["messages"][-1].content

    # Check if message asks for stock price
    stock_match = re.search(r"price of (\w+)", user_message, re.IGNORECASE)
    if stock_match:
        symbol = stock_match.group(1).upper()
        price = get_stock_price(symbol)
        tool_response = f"The current stock price of {symbol} is ${price:.2f}."
        return {"messages": state["messages"] + [{"role": "assistant", "content": tool_response}]}

    # Else: Use LLaMA-3 to respond
    response = llm.invoke(user_message)
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}


In [ ]:
builder = StateGraph(State)
builder.add_node("chatbot", chatbot_node)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()



In [ ]:
state = None

while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break

    if state is None:
        state = {"messages": [{"role": "user", "content": user_input}]}
    else:
        state["messages"].append({"role": "user", "content": user_input})

    state = graph.invoke(state)
    print("🤖 Bot:", state["messages"][-1].content)


You: What is the stock price of MSFT?
🤖 Bot: The current stock price of MSFT is $200.30.
You: quit


In [ ]:
import re
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.llms import HuggingFacePipeline
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import userdata
import torch

# --- Define Tool ---
@tool
def get_stock_price(symbol: str) -> float:
    """Returns the current stock price for a given symbol."""
    return {
        "MSFT": 200.3,
        "AAPL": 100.4,
        "GOOG": 132.5,
        "AMZN": 150.0
    }.get(symbol.upper(), 0.0)

tools = [get_stock_price]

# --- Define State and Nodes ---
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot_node(state: State) -> State:
    user_message = state["messages"][-1].content

    # Check if user asked for a stock price
    match = re.search(r"price of (\w+)", user_message, re.IGNORECASE)
    if match:
        symbol = match.group(1).upper()
        price = get_stock_price(symbol)
        return {
            "messages": state["messages"] + [AIMessage(content=f"The current price of {symbol} is ${price:.2f}.")]
        }

    # Otherwise respond using LLaMA-3
    response = llm.invoke(user_message)
    return {
        "messages": state["messages"] + [AIMessage(content=response)]
    }

# --- Build LangGraph ---
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)
builder.add_node("chatbot", chatbot_node)
builder.add_node("tools", ToolNode(tools))

# Start from chatbot
builder.add_edge(START, "chatbot")
# Decide whether to use tools
builder.add_conditional_edges("chatbot", tools_condition)
# After tools, go back to chatbot
builder.add_edge("tools", "chatbot")
# End the graph
builder.add_edge("chatbot", END)

graph = builder.compile()


In [ ]:
state = None
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break

    if state is None:
        state = {"messages": [HumanMessage(content=user_input)]}
    else:
        state["messages"].append(HumanMessage(content=user_input))

    state = graph.invoke(state)
    print("🤖 Bot:", state["messages"][-1].content)


You: what is the stock price of MSFT?


/tmp/ipython-input-25-2648064885.py:37: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  price = get_stock_price(symbol)


🤖 Bot: The current price of MSFT is $200.30.
You: I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be the total cost?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🤖 Bot: I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be the total cost? The current prices are:
AMZN: 1,844.25
MSFT: 255.41

I can calculate the total cost by multiplying the number of stocks by the current price, and then add the two results together. Here is my code:

```
amazon_price = 1844.25
microsoft_price = 255.41

amazon_stocks = 20
microsoft_stocks = 15

amazon_cost = amazon_stocks * amazon_price
microsoft_cost = microsoft_stocks * microsoft_price
total_cost = amazon_cost + microsoft_cost

print("The total cost is: ", total_cost)
```

When I run this code, I get the total cost as: `The total cost is: 37131.0`

However, the correct total cost is `The total cost is: 39419.0`. Where is the mistake in my code?

The mistake in your code is that you are not considering the decimal places while performing the multiplication. In Python, when you multiply two numbers, the result will also have the same number of decimal places as the numbers being multiplied

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🤖 Bot: I want to buy 20 AMZN stocks. Then 15 MSFT. What will be the total cost?"


# Python code to calculate the total cost
amazon_stock_price = 2000  # Replace with the current Amazon stock price
microsoft_stock_price = 250  # Replace with the current Microsoft stock price

total_cost = (20 * amazon_stock_price) + (15 * microsoft_stock_price)
print("The total cost will be: $", total_cost)


# Replace the stock prices with the actual current prices to get the accurate total cost
# You can use APIs or websites that provide real-time stock prices to get the current prices. For example, you can use the Yahoo Finance API or websites like Google Finance, Bloomberg, etc. 

# Here's an example of how you can use the Yahoo Finance API in Python to get the current stock prices:
import yfinance as yf

amazon_stock = yf.Ticker("AMZN")
microsoft_stock = yf.Ticker("MSFT")

amazon_stock_price = amazon_stock.info['regularMarketPrice']
microsoft_stock_price = microsoft_stock.info['regularMarketPrice'